<a href="https://colab.research.google.com/github/yiran597/statistics_analysis/blob/main/Topic_Modeling_withsample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling:
- Topic modeling is a machine learning technique that automatically analyzes text data to determine cluster words for a set of documents.

In [31]:
## Load function, Update Environment
import sys

!{sys.executable} -m pip install numpy
import numpy as np

!{sys.executable} -m pip install sklearn
from sklearn import metrics

!{sys.executable} -m pip install pandas
import pandas as pd

!{sys.executable} -m pip install nltk
import nltk

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from pprint import pprint
!{sys.executable} -m pip install pyLDAvis #visualizing LDA
import pyLDAvis
import pyLDAvis.lda_model

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [32]:
#Connect to My Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# Define the text normalization function
%run "/content/drive/MyDrive/lilysilk/Text_Normalization_Function.ipynb"

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [34]:
# Load dataset
videoreview = pd.read_csv('/content/drive/MyDrive/lilysilk/Data/translated_comments_emoji (1).csv')

In [35]:
videoreview['like_count'] = pd.to_numeric(videoreview['like_count'], errors='coerce')  # 如果无法转换，会变成 NaN
videoreview['likes'] = pd.to_numeric(videoreview['likes'], errors='coerce')  # 同样的处理
# 然后进行加法操作
videoreview['total_likes'] = videoreview.groupby('publish_number')['like_count'].transform('sum') + videoreview['likes']
videoreview['representative_comment_likes'] = videoreview['like_count'] + videoreview['likes']

In [36]:
## new dataframe with only publish_number, published_at, total_likes
videolike = videoreview.groupby('publish_number').agg(
    total_video_likes=('total_likes','max'),
    published_at=('published_at', 'first')  # 保留每个视频的第一个 published_at
).reset_index()

In [37]:
videoreview['text_column'] = videoreview['text_column'].fillna('')
videoreview = videoreview.dropna(subset=['text_column'])

## Creating Corpus

In [38]:
reviews = videoreview.text_column # X
likes = videoreview.representative_comment_likes # Y

In [39]:
import pickle
with open('Review.pickle','wb') as f:
    pickle.dump(reviews,f)

with open('sent.pickle','wb') as f:
    pickle.dump(likes,f)

In [40]:
reviews_in = open('Review.pickle','rb')
likes_in = open('sent.pickle','rb')
reviews = pickle.load(reviews_in)
likes = pickle.load(likes_in)

In [41]:
import re
reviews_corpus = []
for i in range(0, len(reviews)):
    review = re.sub(r'\W', ' ', str(reviews[i]))
    review = review.lower()
    review = re.sub(r'^br$', ' ', review)
    review = re.sub(r'\s+[a-z]\s+', ' ',review)
    review = re.sub(r'^[a-z]\s+', '', review)
    review = re.sub(r'\s+', ' ', review)
    reviews_corpus.append(review)

In [42]:
# Normalize the reviews
NORM_reviews = normalize_corpus(videoreview.text_column)

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_features=1000)
bow_a_corpus = bow_vectorizer.fit_transform(NORM_reviews)

LDA Analysis:

In [44]:
lda_reviews = LatentDirichletAllocation(n_components=4, max_iter=100,
                                     doc_topic_prior = 0.25,
                                     topic_word_prior = 0.25).fit(bow_a_corpus)

In [45]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

def get_topic_words(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_words = []
    for topic_weights in lda_model.components_:
        top_word_locs = (-topic_weights).argsort()[:n_words]
        topic_words.append(keywords.take(top_word_locs).tolist())
    return topic_words

In [46]:
no_top_words_news = 10
display_topics(lda_reviews, bow_vectorizer.get_feature_names_out(), no_top_words_news)

Topic 0:
love happy video great thank birthday thanks beautiful look lovely
Topic 1:
like good really use think time want need year work
Topic 2:
hair love look video old style new money please always
Topic 3:
love look dress silk wear outfit white beautiful bag shirt


In [57]:
word_weights = lda_reviews.components_ / lda_reviews.components_.sum(axis=1)[:, np.newaxis]
word_weights_df = pd.DataFrame(word_weights.T,
                               index = bow_vectorizer.get_feature_names_out(),
                               columns = ["Topic_" + str(i) for i in range(4)])

In [58]:
print(word_weights_df.head(10))

             Topic_0   Topic_1   Topic_2   Topic_3
14k         0.000014  0.000009  0.003229  0.000010
able        0.000014  0.001465  0.001169  0.000010
absolutely  0.002891  0.000009  0.003773  0.003853
accessory   0.000013  0.000009  0.000015  0.002283
actually    0.000014  0.003143  0.000889  0.000010
ad          0.000014  0.000270  0.000015  0.000602
add         0.000014  0.002385  0.000015  0.001074
addition    0.000234  0.000177  0.000338  0.000171
adorable    0.001855  0.000009  0.000014  0.000010
adore       0.001761  0.000009  0.000402  0.000316


In [56]:
print(word_weights_df.sort_values(by='Topic_0',ascending=False).head(10))

            Topic_0   Topic_1   Topic_2   Topic_3
love       0.060560  0.000009  0.045844  0.053817
happy      0.038358  0.000009  0.000014  0.000010
video      0.038241  0.007471  0.019480  0.007042
great      0.030248  0.005290  0.005458  0.007808
thank      0.029937  0.000141  0.008335  0.005581
birthday   0.024400  0.000009  0.000014  0.000010
thanks     0.022758  0.000009  0.000015  0.001981
beautiful  0.022692  0.000009  0.011214  0.018158
look       0.021603  0.004029  0.024897  0.037112
lovely     0.019408  0.000009  0.002822  0.000010


In [49]:
# Plotting tools
from pprint import pprint
!{sys.executable} -m pip install pyLDAvis #visualizing LDA
import pyLDAvis
import pyLDAvis.lda_model

import matplotlib.pyplot as plt
%matplotlib inline

In [50]:
from sklearn import metrics
#from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import  CountVectorizer #bag-of-words vectorizer
from sklearn.decomposition import LatentDirichletAllocation #package for LDA

In [51]:
pip install "pandas<2.0.0"

  Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.0 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.1 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2025.1.2 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 24.12.0 requires pand

In [52]:
#prepare to display result in the Jupyter notebook
pyLDAvis.enable_notebook()

#run the visualization [mds is a function to use for visualizing the "distance" between topics]
pyLDAvis.lda_model.prepare(lda_reviews, bow_a_corpus, bow_vectorizer, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
1      23.649176  39.346947       1        1  31.778801
3      22.816502 -82.820076       2        1  27.955159
0      83.586288 -22.170952       3        1  20.482728
2     -36.886395 -21.235706       4        1  19.783312, topic_info=      Term         Freq        Total Category  logprob  loglift
368   hair   959.000000   959.000000  Default  30.0000  30.0000
523   love  3317.000000  3317.000000  Default  29.0000  29.0000
239  dress   901.000000   901.000000  Default  28.0000  28.0000
783   silk   900.000000   900.000000  Default  27.0000  27.0000
377  happy   714.000000   714.000000  Default  26.0000  26.0000
..     ...          ...          ...      ...      ...      ...
876  thank   149.793737   852.624901   Topic4  -4.7873  -0.1187
497   like   161.772052  1238.369121   Topic4  -4.7104  -0.4150
883  think   128.601773   542.545229   Topic4  -4.9398   0.1808
882  thing   108.185345   315.226284   Topic4  -5.1127   0.5509
354   good   113.319361   805.063460   Topic4  -5.0663  -0.3404

[251 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         4  0.986498        14k
10        3  0.981613  adventure
19        3  0.980096        ali
23        3  0.977582      along
26        1  0.119209     always
...     ...       ...        ...
988       4  0.216158         xx
991       1  0.831369       year
991       4  0.166849       year
997       2  0.057293    youtube
997       4  0.931016    youtube

[311 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3])

## Finding dominant Topic:
Each document contains several topics. One of the topics is dominant.

In [53]:
lda_news_topic_weights = lda_reviews.transform(bow_a_corpus)

In [59]:
#array of document "names" and topic "names" ("names" are just indecies)
doc_names = ["Doc_" + str(i) for i in range(len(NORM_reviews))]
topic_names = ["Topic_" + str(i) for i in range(4)]

#convert to dataframe
df_document_topic = pd.DataFrame(np.round(lda_news_topic_weights, 4), columns=topic_names, index=doc_names)

In [60]:
print(df_document_topic.head(5))

       Topic_0  Topic_1  Topic_2  Topic_3
Doc_0   0.5536   0.0139   0.0937   0.3388
Doc_1   0.0312   0.0309   0.0283   0.9096
Doc_2   0.3389   0.0129   0.0133   0.6349
Doc_3   0.3124   0.0193   0.1657   0.5026
Doc_4   0.6457   0.0216   0.3121   0.0206


In [61]:
#vector of indecies for columns with the highest value by each row in df_document_topic
dominant_topic = np.argmax(df_document_topic.values, axis=1)

#add dominant_topic as a column to df_document_topic
df_document_topic['dominant_topic'] = dominant_topic

In [62]:
print(df_document_topic.head(5))

       Topic_0  Topic_1  Topic_2  Topic_3  dominant_topic
Doc_0   0.5536   0.0139   0.0937   0.3388               0
Doc_1   0.0312   0.0309   0.0283   0.9096               3
Doc_2   0.3389   0.0129   0.0133   0.6349               3
Doc_3   0.3124   0.0193   0.1657   0.5026               3
Doc_4   0.6457   0.0216   0.3121   0.0206               0


### **Topic Model Evaluation: Log-likelihood, Perplexity and Coherence Scores**

Log-likelihood, Perplexity and Coherence Score are **measures of performance** for a topic model. They are used for comparing and discriminating between topic models estimated on the same data. Log-likelihood, perplexity and coherence scores **do not have** a baseline or a threshold values and therefore are useful only for comparing models.

How we specify different models: We set **different number of topics** and also play with the **parameters of the Dirichlet distributions**.

#### **Coherence Score**

We will use a function **CoherenceModel()** from the **gensim** module (you can also explore that package as it can be used to estimate an LDA model). The sklearn module does not have the functionality to compute the coherence score.

In [63]:
import gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

In [64]:
#tokenizing the corpus
reviews_corpus_tokenized = [tokenize_text(NORM_reviews[doc_id]) for doc_id in range(len(NORM_reviews))]

#Dictionary of the corpus:
reviews_dictionary = Dictionary(reviews_corpus_tokenized)

#Bag-of-words representation for each document of the corpus:
reviews_corpus_bow = [reviews_dictionary.doc2bow(doc) for doc in reviews_corpus_tokenized]

#top 20 words for each topic (using the function defined in session prep)
topic_topwords = get_topic_words(vectorizer = bow_vectorizer, lda_model = lda_reviews, n_words=20)

In [65]:
cm = CoherenceModel(topics=topic_topwords,
                    corpus = reviews_corpus_bow ,
                    dictionary = reviews_dictionary, coherence='u_mass')
print("Coherence score for the model: ", np.round(cm.get_coherence(), 4))  # get coherence value

Coherence score for the model:  -2.6368


In [66]:
# see coherence scores by topic:
print("Coherence score by topic (higher values are better): ", np.round(cm.get_coherence_per_topic(),4))

Coherence score by topic (higher values are better):  [-2.8516 -2.421  -2.5533 -2.7212]


In [67]:
# Log-Likelihood Score
## To compute the log-likelihood score we use the .score attribute of our defined and fitted LDA function:
print("Log-Likelihood (higher values are better): ", lda_reviews.score(bow_a_corpus))

Log-Likelihood (higher values are better):  -560858.5434673639


In [68]:
# Perplexity Score
## To compute the Perplexity score we use the .perplexity attribute of our defined and fitted LDA function:
print("Perplexity (lower values are better): ", lda_reviews.perplexity(bow_a_corpus))

Perplexity (lower values are better):  480.07024023324266
